# Train a domain classifier on the [semantic scholar dataset](https://api.semanticscholar.org/corpus)

## Part 1: obtain clean data

This can be achieved by downloading a public dataset (such as in this case), or by having a Data Engineer setup and give us access to the organisations's data.

1. **Extract**: Download the semantic scholar dataset from a public S3 bucket
2. **Transform**: Project it to only keep the necessary components (text and labels), clean the textual content using `great_ai.utilities.clean`
3. **Load**: Upload the dataset (or a part of it) to a shared infrastructure using `great_ai.LargeFile` with its S3 backend

> We will speed up the processing using `great_ai.utilities.parallel_map`

In [1]:
import json
from typing import List, Tuple
import urllib.request
from itertools import chain
import gzip
from great_ai.utilities.clean import clean
from great_ai.utilities.parallel_map import parallel_map
from great_ai.utilities.language import is_english, predict_language
from great_ai import configure, LargeFile


DATASET_KEY = "semantic-scholar-dataset-small"
MAX_FILE_COUNT = 12
configure()

2022-05-27 12:12:27,324 |     INFO | Environment variable ENVIRONMENT is not set, defaulting to development mode
2022-05-27 12:12:27,324 |  WARNING | Running in development mode ‼️
2022-05-27 12:12:27,325 |     INFO | Options: configured ✅


In [2]:
# Download the list of available chunks

manifest = (
    urllib.request.urlopen(
        "https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2022-02-01/manifest.txt"
    )
    .read()
    .decode()
)
chunks = manifest.split()[:MAX_FILE_COUNT]

In [3]:
# Extract, Transform


def process_chunk(chunk_key: str) -> List[Tuple[str, List[str]]]:
    response = urllib.request.urlopen(
        "https://s3-us-west-2.amazonaws.com/ai2-s2-research-public/open-corpus/2022-02-01/"
        + chunk_key
    )  # returns a gzipped JSON Lines file
    decompressed = gzip.decompress(response.read())
    decoded = decompressed.decode()
    chunk = [json.loads(line) for line in decoded.split("\n") if line]

    all = [
        # Create pairs of `(text, [domains])`
        # The text is cleaned to remove PDF extraction, web scraping, and other common artifacts
        (
            clean(
                f'{c["title"]} {c["paperAbstract"]} {c["journalName"]} {c["venue"]}',
                convert_to_ascii=True,
            ),
            c["fieldsOfStudy"],
        )
        for c in chunk
    ]

    return [
        (content, domains)
        for content, domains in all
        if (domains and content and is_english(predict_language(content)))
    ]


clean_chunks = parallel_map(process_chunk, chunks)

2022-05-27 12:12:28,639 |     INFO | Starting parallel map (concurrency: 12, chunk size: 1)


  0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
# Load
# Using the LargeFile GreatAI utility, the data will be made available on S3 (and in our local cache)

with LargeFile(DATASET_KEY, "w", keep_last_n=1) as f:
    json.dump(list(chain(*clean_chunks)), f)

2022-05-27 12:22:35,833 |     INFO | Fetching online versions of semantic-scholar-dataset-small
2022-05-27 12:22:36,133 |     INFO | Found versions: [2, 3]
2022-05-27 12:22:39,542 |     INFO | Copying file for semantic-scholar-dataset-small-4
2022-05-27 12:22:39,843 |     INFO | Compressing semantic-scholar-dataset-small-4
2022-05-27 12:22:54,990 |     INFO | Uploading semantic-scholar-dataset-small-4 to S3 from /tmp/large-file-l9z1mo61
2022-05-27 12:23:19,988 |     INFO | Uploading semantic-scholar-dataset-small-4.tar.gz  8.91/88.72 MB (10.0%)
2022-05-27 12:23:41,914 |     INFO | Uploading semantic-scholar-dataset-small-4.tar.gz 17.83/88.72 MB (20.1%)
2022-05-27 12:24:06,996 |     INFO | Uploading semantic-scholar-dataset-small-4.tar.gz 26.74/88.72 MB (30.1%)
2022-05-27 12:24:30,563 |     INFO | Uploading semantic-scholar-dataset-small-4.tar.gz 35.65/88.72 MB (40.2%)
2022-05-27 12:24:59,623 |     INFO | Uploading semantic-scholar-dataset-small-4.tar.gz 44.56/88.72 MB (50.2%)
2022-05-2